# 차원축소 Assignment1 Tobigs15th 이성범

# ''' ? ''' 이 있는 부분을 채워주세요 

참고하셔야 하는 함수들은 링크 달아드렸으니 들어가서 확인해보세요

무작정 빈칸 채우기가 아니라 꼭 차원축소 과정을 이해하시면서 과제를 진행하시면 좋겠습니다!!!

또 질문은 언제나 환영이니 과제하다가 궁금하신것 있으시면 자유롭게 물어보세요!!

# 1) PCA의 과정 차근차근 밟아보기

In [1]:
#기본 모듈 불러오기
import numpy as np
import numpy.linalg as lin
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
#설명변수 x1, x2, x3
x1 = [95, 91, 66, 94, 68, 63, 12, 73, 93, 51, 13, 70, 63, 63, 97, 56, 67, 96, 75, 6]
x2 = [56, 27, 25, 1, 9, 80, 92, 69, 6, 25, 83, 82, 54, 97, 66, 93, 76, 59, 94, 9]
x3 = [57, 34, 9, 79, 4, 77, 100, 42, 6, 96, 61, 66, 9, 25, 84, 46, 16, 63, 53, 30]

In [3]:
#   설명변수들을 하나의 행렬로
X = np.stack((x1,x2,x3),axis=0)

In [4]:
X = pd.DataFrame(X.T,columns=['x1','x2','x3'])

In [5]:
X

,x1,x2,x3
0,95,56,57
1,91,27,34
2,66,25,9
3,94,1,79
4,68,9,4
5,63,80,77
6,12,92,100
7,73,69,42
8,93,6,6
9,51,25,96


### 1-1) 먼저 PCA를 시작하기 전 데이터 scaling!!!! 매우 중요합니다!

https://datascienceschool.net/03%20machine%20learning/02.01%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%A0%84%EC%B2%98%EB%A6%AC%20%EA%B8%B0%EC%B4%88.html?highlight=%EC%A0%84%EC%B2%98%EB%A6%AC  를 참고하시면 도움이 될거에요

In [6]:
#scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X) #sclaing 후 데이터

StandardScaler을 하는 이유는 PCA의 경우 분산이 큰 쪽으로 차원을 축소하는 경향이 존재하기 때문에 표준화를 통해서 모든 차원의 분산을 1로 통일시켜줘야한다.

In [7]:
X_std

array([[ 1.08573604,  0.02614175,  0.30684189],
       [ 0.93801686, -0.86575334, -0.46445467],
       [ 0.01477192, -0.92726334, -1.30282049],
       [ 1.04880625, -1.66538341,  1.04460382],
       [ 0.08863151, -1.41934339, -1.47049366],
       [-0.09601747,  0.76426183,  0.97753455],
       [-1.97943714,  1.13332186,  1.74883111],
       [ 0.2732805 ,  0.42595679, -0.1961776 ],
       [ 1.01187645, -1.5116084 , -1.40342439],
       [-0.53917504, -0.92726334,  1.61469258],
       [-1.94250735,  0.85652683,  0.44098042],
       [ 0.16249111,  0.82577183,  0.60865359],
       [-0.09601747, -0.03536825, -1.30282049],
       [-0.09601747,  1.28709688, -0.76626636],
       [ 1.15959564,  0.33369178,  1.21227698],
       [-0.35452606,  1.16407687, -0.06203907],
       [ 0.05170172,  0.64124181, -1.06807806],
       [ 1.12266584,  0.11840676,  0.50804969],
       [ 0.3471401 ,  1.19483187,  0.17270336],
       [-2.20101593, -1.41934339, -0.5985932 ]])

In [8]:
features = X_std.T

In [9]:
features

array([[ 1.08573604,  0.93801686,  0.01477192,  1.04880625,  0.08863151,
        -0.09601747, -1.97943714,  0.2732805 ,  1.01187645, -0.53917504,
        -1.94250735,  0.16249111, -0.09601747, -0.09601747,  1.15959564,
        -0.35452606,  0.05170172,  1.12266584,  0.3471401 , -2.20101593],
       [ 0.02614175, -0.86575334, -0.92726334, -1.66538341, -1.41934339,
         0.76426183,  1.13332186,  0.42595679, -1.5116084 , -0.92726334,
         0.85652683,  0.82577183, -0.03536825,  1.28709688,  0.33369178,
         1.16407687,  0.64124181,  0.11840676,  1.19483187, -1.41934339],
       [ 0.30684189, -0.46445467, -1.30282049,  1.04460382, -1.47049366,
         0.97753455,  1.74883111, -0.1961776 , -1.40342439,  1.61469258,
         0.44098042,  0.60865359, -1.30282049, -0.76626636,  1.21227698,
        -0.06203907, -1.06807806,  0.50804969,  0.17270336, -0.5985932 ]])

### 1-2) 다음으로 공분산 행렬을 구합니다

https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html 를 참고하시면 도움이 될거에요

In [10]:
cov_matrix = np.cov(features)

In [11]:
cov_matrix

array([[ 1.05263158, -0.2037104 , -0.12079228],
       [-0.2037104 ,  1.05263158,  0.3125801 ],
       [-0.12079228,  0.3125801 ,  1.05263158]])

### 1-3) 이제 고유값과 고유벡터를 구합니다

실습코드를 참고해주세요

In [12]:
#Eigenvalue와 Eigenvector 함께 출력
eigenvalues_and_eigenvectors = lin.eig(cov_matrix)
eigenvalues_and_eigenvectors

(array([1.48756162, 0.94435407, 0.72597904]),
 array([[ 0.47018528, -0.85137353, -0.23257022],
        [-0.64960236, -0.15545725, -0.74421087],
        [-0.59744671, -0.50099516,  0.62614797]]))

In [13]:
# Eigenvalue
eigenvalues = eigenvalues_and_eigenvectors[0]
# Eigenvector
eigenvectors = eigenvalues_and_eigenvectors[1]

In [14]:
print(eigenvalues)
print(eigenvectors)

[1.48756162 0.94435407 0.72597904]
[[ 0.47018528 -0.85137353 -0.23257022]
 [-0.64960236 -0.15545725 -0.74421087]
 [-0.59744671 -0.50099516  0.62614797]]


In [15]:
# 3행 3열이 0으로 채워진 배열 생성
mat = np.zeros((3,3))

In [16]:
mat

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [17]:
mat[0][0] = eigenvalues[0]
mat[1][1] = eigenvalues[1]
mat[2][2] = eigenvalues[2]

In [18]:
# 고유값을 대각원소로 하는 대각행렬
mat

array([[1.48756162, 0.        , 0.        ],
       [0.        , 0.94435407, 0.        ],
       [0.        , 0.        , 0.72597904]])

### 1-4) 고유값 분해: 고유값 분해의 곱으로 원래 공분산 행렬을 구해봅니다

https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html 를 참고해주세요

A = 𝑃Λ𝑃`
이 식 기억나죠?

Λ -> 고유값을 대각원소로 하는 대각행렬

𝑃  -> 고유 벡터의 열로 구성된 행렬

𝑃` -> 𝑃의 역행렬

Eigenvector의 경우 직교행렬이기 때문에 Eigenvector를 전치한 행렬과 Eigenvector의 역행렬이 동일하다.

In [19]:
# eigenvectors를 전치한 행렬
eigenvectors.T

array([[ 0.47018528, -0.64960236, -0.59744671],
       [-0.85137353, -0.15545725, -0.50099516],
       [-0.23257022, -0.74421087,  0.62614797]])

In [20]:
# eigenvectors의 역행렬
lin.inv(eigenvectors)

array([[ 0.47018528, -0.64960236, -0.59744671],
       [-0.85137353, -0.15545725, -0.50099516],
       [-0.23257022, -0.74421087,  0.62614797]])

In [21]:
np.dot(np.dot(eigenvectors, mat), lin.inv(eigenvectors))

array([[ 1.05263158, -0.2037104 , -0.12079228],
       [-0.2037104 ,  1.05263158,  0.3125801 ],
       [-0.12079228,  0.3125801 ,  1.05263158]])

In [22]:
np.dot(np.dot(eigenvectors, mat), eigenvectors.T)

array([[ 1.05263158, -0.2037104 , -0.12079228],
       [-0.2037104 ,  1.05263158,  0.3125801 ],
       [-0.12079228,  0.3125801 ,  1.05263158]])

In [23]:
cov_matrix

array([[ 1.05263158, -0.2037104 , -0.12079228],
       [-0.2037104 ,  1.05263158,  0.3125801 ],
       [-0.12079228,  0.3125801 ,  1.05263158]])

1-2에서 구했던 공분산과 동일하다는 것을 알 수 있다.

### 1-5) 마지막으로 고유 벡터 축으로 값을 변환합니다


https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html

In [24]:
def new_coordinates(X, eigenvectors):
  for i in range(eigenvectors.shape[0]):
    if i == 0:
      new = [X.dot(eigenvectors.T[i])]
    else:
      new = np.concatenate((new, [X.dot(eigenvectors.T[i])]), axis=0)
  return new.T

# 모든 고유 벡터 축으로 데이터를 projection한 값입니다

In [25]:
new_coordinates(X_std,eigenvectors)

# 새로운 축으로 변환되어 나타난 데이터들입니다

array([[ 0.31019368, -1.08215716, -0.07983642],
       [ 1.28092404, -0.43132556,  0.13533091],
       [ 1.38766381,  0.78428014, -0.12911446],
       [ 0.95087515, -1.15737142,  1.6495519 ],
       [ 1.84222365,  0.88189889,  0.11493111],
       [-1.12563709, -0.52680338,  0.06564012],
       [-2.71174416,  0.63290138,  0.71195473],
       [-0.03100441, -0.20059783, -0.50339479],
       [ 2.29618509,  0.07661447,  0.01087174],
       [-0.61585248, -0.205764  ,  1.82651199],
       [-1.73320252,  1.29971699,  0.09045178],
       [-0.82366049, -0.57164535, -0.27123176],
       [ 0.75619512,  0.73995175, -0.76710616],
       [-0.42344386,  0.26555394, -1.41533681],
       [-0.39581307, -1.64646874,  0.24104031],
       [-0.88581498,  0.15195119, -0.82271209],
       [ 0.24587691,  0.39139878, -1.15801831],
       [ 0.14741103, -1.22874561, -0.03110396],
       [-0.7161265 , -0.56781471, -0.86180345],
       [ 0.24475107,  2.39442622,  1.19337361]])

# 2) PCA를 구현해보기

위의 과정을 하나의 함수로 표현해주세요!

1)에서 진행한 과정을 그대로 가져오면 됩니다..! , 

위에서 만든 함수(new_coordinate)도 가져오셔서 사용해도 좋아요!

+하지만 이 함수는 number이라는 인자를 받아서 pca에서 주성분 개수를 고려하도록 합니다!

고유값이 큰 축 순서로 정렬하는 과정 + 주성분 개수 number을 고려해야한다는 점을 유의해서 함수를 정의해주세요


In [26]:
eigenvalues

array([1.48756162, 0.94435407, 0.72597904])

In [27]:
eigenvalues.argsort()[::-1]

array([0, 1, 2])

In [28]:
eigenvalues[eigenvalues.argsort()]

array([0.72597904, 0.94435407, 1.48756162])

In [29]:
from sklearn.preprocessing import StandardScaler

def MYPCA(X, number):
  scaler = StandardScaler()
  X_std = scaler.fit_transform(X) #sclaing 후 데이터

  features = X_std.T

  cov_matrix = np.cov(features)

  eigenvalues_and_eigenvectors = lin.eig(cov_matrix)
  # Eigenvalue
  eigenvalues = eigenvalues_and_eigenvectors[0]
  # Eigenvector
  eigenvectors = eigenvalues_and_eigenvectors[1]

  new_coordinate = new_coordinates(X_std, eigenvectors)

  # 고유값이 큰 축 순서의 인덱스를 반환
  idx = eigenvalues.argsort()[::-1].tolist()

  if number > len(idx):
    print(f'선택된 주성분의 요소가 Eigenvalue의 개수보다 많습니다. {len(idx)}이하로 다시 설정해주세요')
    return

  for i in range(number):
    if i == 0:
      new = [new_coordinate[:, idx[i]]]
    else:
      new = np.concatenate((new, [new_coordinate[:, idx[i]]]), axis=0)

  return new.T #새로운 축으로 변환되어 나타난 데이터 return

In [30]:
MYPCA(X,4)

선택된 주성분의 요소가 Eigenvalue의 개수보다 많습니다. 3이하로 다시 설정해주세요


In [31]:
MYPCA(X,3)

# 새로운 축으로 잘 변환되어서 나타나나요? 네
# 위에서 했던 PCA랑은 차이가 있을 수 있어요 
# 왜냐하면 위에서는 고유값이 큰 축 순서로 정렬을 안했었거든요

array([[ 0.31019368, -1.08215716, -0.07983642],
       [ 1.28092404, -0.43132556,  0.13533091],
       [ 1.38766381,  0.78428014, -0.12911446],
       [ 0.95087515, -1.15737142,  1.6495519 ],
       [ 1.84222365,  0.88189889,  0.11493111],
       [-1.12563709, -0.52680338,  0.06564012],
       [-2.71174416,  0.63290138,  0.71195473],
       [-0.03100441, -0.20059783, -0.50339479],
       [ 2.29618509,  0.07661447,  0.01087174],
       [-0.61585248, -0.205764  ,  1.82651199],
       [-1.73320252,  1.29971699,  0.09045178],
       [-0.82366049, -0.57164535, -0.27123176],
       [ 0.75619512,  0.73995175, -0.76710616],
       [-0.42344386,  0.26555394, -1.41533681],
       [-0.39581307, -1.64646874,  0.24104031],
       [-0.88581498,  0.15195119, -0.82271209],
       [ 0.24587691,  0.39139878, -1.15801831],
       [ 0.14741103, -1.22874561, -0.03110396],
       [-0.7161265 , -0.56781471, -0.86180345],
       [ 0.24475107,  2.39442622,  1.19337361]])

# 3) sklearn이랑 비교를 해보기

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html 를 참고하시면 도움이 될거에요

sklearn의 PCA는 고유값 분해(eigenvalue-decomposition)가 아닌 특이값 분해(SVD, Singular Value Decomposition)의 방식으로 차원을 축소한다.

이러한 분해 방식의 차이에 의하여 우리가 구한 PCA와 사이킷런의 PCA의 부호가 다르다는 것을 확인 할 수 있다.

부호가 다르다는 것은 벡터의 방향이 반대라는 의미일뿐이며 값이 같음으로 주성분 벡터가 구성하는 축이 동일하기 때문에 두 방식의 결과의 차이는 큰 의미가 없다.

사이킷런에서는 고유값 분해가 아닌 특이값 분해를 사용하는 이유는 특이값 분해의 경우 행렬이 정방행렬에 관계없이 모든 m x n 행렬에 대해 적용 가능하기 때문이며 고유값 분해의 경우 정방행렬의 경우에만 사용 가능하기 때문에 차원 축소의 폭을 더 넓히고자 특이값 분해를 사용하는 것으로 유추해볼 수 있다.

참고

1.https://excelsior-cjh.tistory.com/167

2.https://darkpgmr.tistory.com/106

In [32]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [33]:
pca.fit_transform(X_std)

array([[-0.31019368, -1.08215716, -0.07983642],
       [-1.28092404, -0.43132556,  0.13533091],
       [-1.38766381,  0.78428014, -0.12911446],
       [-0.95087515, -1.15737142,  1.6495519 ],
       [-1.84222365,  0.88189889,  0.11493111],
       [ 1.12563709, -0.52680338,  0.06564012],
       [ 2.71174416,  0.63290138,  0.71195473],
       [ 0.03100441, -0.20059783, -0.50339479],
       [-2.29618509,  0.07661447,  0.01087174],
       [ 0.61585248, -0.205764  ,  1.82651199],
       [ 1.73320252,  1.29971699,  0.09045178],
       [ 0.82366049, -0.57164535, -0.27123176],
       [-0.75619512,  0.73995175, -0.76710616],
       [ 0.42344386,  0.26555394, -1.41533681],
       [ 0.39581307, -1.64646874,  0.24104031],
       [ 0.88581498,  0.15195119, -0.82271209],
       [-0.24587691,  0.39139878, -1.15801831],
       [-0.14741103, -1.22874561, -0.03110396],
       [ 0.7161265 , -0.56781471, -0.86180345],
       [-0.24475107,  2.39442622,  1.19337361]])

In [34]:
-MYPCA(X,3)

array([[-0.31019368,  1.08215716,  0.07983642],
       [-1.28092404,  0.43132556, -0.13533091],
       [-1.38766381, -0.78428014,  0.12911446],
       [-0.95087515,  1.15737142, -1.6495519 ],
       [-1.84222365, -0.88189889, -0.11493111],
       [ 1.12563709,  0.52680338, -0.06564012],
       [ 2.71174416, -0.63290138, -0.71195473],
       [ 0.03100441,  0.20059783,  0.50339479],
       [-2.29618509, -0.07661447, -0.01087174],
       [ 0.61585248,  0.205764  , -1.82651199],
       [ 1.73320252, -1.29971699, -0.09045178],
       [ 0.82366049,  0.57164535,  0.27123176],
       [-0.75619512, -0.73995175,  0.76710616],
       [ 0.42344386, -0.26555394,  1.41533681],
       [ 0.39581307,  1.64646874, -0.24104031],
       [ 0.88581498, -0.15195119,  0.82271209],
       [-0.24587691, -0.39139878,  1.15801831],
       [-0.14741103,  1.22874561,  0.03110396],
       [ 0.7161265 ,  0.56781471,  0.86180345],
       [-0.24475107, -2.39442622, -1.19337361]])

In [35]:
MYPCA(X,3)

array([[ 0.31019368, -1.08215716, -0.07983642],
       [ 1.28092404, -0.43132556,  0.13533091],
       [ 1.38766381,  0.78428014, -0.12911446],
       [ 0.95087515, -1.15737142,  1.6495519 ],
       [ 1.84222365,  0.88189889,  0.11493111],
       [-1.12563709, -0.52680338,  0.06564012],
       [-2.71174416,  0.63290138,  0.71195473],
       [-0.03100441, -0.20059783, -0.50339479],
       [ 2.29618509,  0.07661447,  0.01087174],
       [-0.61585248, -0.205764  ,  1.82651199],
       [-1.73320252,  1.29971699,  0.09045178],
       [-0.82366049, -0.57164535, -0.27123176],
       [ 0.75619512,  0.73995175, -0.76710616],
       [-0.42344386,  0.26555394, -1.41533681],
       [-0.39581307, -1.64646874,  0.24104031],
       [-0.88581498,  0.15195119, -0.82271209],
       [ 0.24587691,  0.39139878, -1.15801831],
       [ 0.14741103, -1.22874561, -0.03110396],
       [-0.7161265 , -0.56781471, -0.86180345],
       [ 0.24475107,  2.39442622,  1.19337361]])